#Anchors for Heart Disease dataset:

In [ ]:
!pip install alibi

     |████████████████████████████████| 225kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 327kB 8.5MB/s 
     |████████████████████████████████| 204kB 8.4MB/s 
     |████████████████████████████████| 28.0MB 136kB/s 
  Created wheel for shap: filename=shap-0.36.0-cp36-cp36m-linux_x86_64.whl size=456464 sha256=28472003706081b3d670020ae68a640e76b39c3dab23fadb0aab4c401868518c
  Stored in directory: /root/.cache/pip/wheels/fb/15/e1/8f61106790da27e0765aaa6e664550ca2c50ea339099e799f4
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052144 sha256=ed3239aad7f56ec64f23101f2812fbea20405dddca59f663349fdd08d7ac4dae
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb6235c56d014d224bca760d15d7cbdd820813085ffcd35d
Successfully built shap spacy-lookups-data


In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.3.0
Eager execution enabled:  False


In [ ]:
df = pd.read_csv('/content/heartu.csv')
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [ ]:
df.shape

(297, 14)

In [ ]:
feature_names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
data = df[feature_names]
target = df.condition

Splitting the data into training and testing set

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data,target, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(C=0.01,solver='liblinear',max_iter=500)
model.fit(x_train,y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Apllying AnchorTabular method 

In [ ]:
from alibi.explainers import AnchorTabular
predict_func=lambda x:model.predict_proba(x)
explainer=AnchorTabular(predict_func,feature_names)

In [ ]:
model.predict_proba(x_test)

array([[0.58247803, 0.41752197],
       [0.77450519, 0.22549481],
       [0.75681729, 0.24318271],
       [0.18091057, 0.81908943],
       [0.86033967, 0.13966033],
       [0.11171929, 0.88828071],
       [0.40723555, 0.59276445],
       [0.19783263, 0.80216737],
       [0.11024544, 0.88975456],
       [0.18950502, 0.81049498],
       [0.56097183, 0.43902817],
       [0.91997135, 0.08002865],
       [0.64081597, 0.35918403],
       [0.47946202, 0.52053798],
       [0.92404139, 0.07595861],
       [0.19096939, 0.80903061],
       [0.78200054, 0.21799946],
       [0.41555401, 0.58444599],
       [0.20363026, 0.79636974],
       [0.53557304, 0.46442696],
       [0.86416571, 0.13583429],
       [0.60047315, 0.39952685],
       [0.19753243, 0.80246757],
       [0.19349917, 0.80650083],
       [0.81447944, 0.18552056],
       [0.77601218, 0.22398782],
       [0.60509221, 0.39490779],
       [0.27504511, 0.72495489],
       [0.54251071, 0.45748929],
       [0.62958067, 0.37041933],
       [0.

In [ ]:
x_train=x_train.to_numpy()
x_test=x_test.to_numpy()

In [ ]:
explainer.fit(x_train)

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

In [ ]:
from sklearn.metrics import jaccard_similarity_score
yjc = model.predict(x_test)
jaccard_similarity_score(y_test, yjc)

jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.


0.76

Taking an instance where the person has no disease

In [ ]:
inst=1
target_label=['no heart disease','heart disease']
print('Person has',target_label[explainer.predictor(x_test[inst].reshape(1, -1))[0]])
anchor=explainer.explain(x_test[inst])
print('Anchor generated feature(/s)',anchor.anchor)

Person has no heart disease
Anchor generated feature(/s) ['thalach > 138.00', 'thal <= 0.00', 'age <= 48.00']


Here, we can see that as the person's maximum heart rate is 152 (which is greater than 138) and thal value is 0 'normal' and age is less than 48, the above features act as anchors for the patient and deduce that the person has no heart disease 

Taking another instance where the person has a heart disease

In [ ]:
print('Person has',target_label[explainer.predictor(x_test[9].reshape(1, -1))[0]])
anchor=explainer.explain(x_test[9])
print('Anchor generated feature(/s)',anchor.anchor)

Person has heart disease
Anchor generated feature(/s) ['thalach <= 138.00', 'ca > 1.00']


Here, we can see that as the person's maximum heart rate is 131 (which is less than 138) and blood vessels coloured by fluoroscopy are 3 (greater than 1), the above features act as anchors for the patient and deduce that the person has a heart disease 